In [95]:
import json
import re
from IPython.display import display, HTML

from datetime import datetime


In [96]:
with open("Ali-diary-notebook.ipynb", "r") as f:
    notebook_data = json.load(f)


In [97]:
def extract_dates_from_cells(notebook_data):
    dates = set()
    for cell in notebook_data["cells"]:
        if cell["cell_type"] == "markdown":
            cell_content = ''.join(cell["source"])
            cell_dates = re.findall(r"(\d{2}-\d{2}-\d{4})", cell_content)
            dates.update(cell_dates)
    return dates


In [98]:
dates = extract_dates_from_cells(notebook_data)


In [99]:
print(dates)

{'31-03-2023', '29-03-2023', '06-04-2023', '05-04-2023', '02-04-2023', '28-03-2023', '27-03-2023', '26-04-2023', '16-04-2023', '12-04-2023', '23-03-2023', '15-04-2023', '08-04-2023'}


In [100]:
def generate_anchor_tags(notebook_data, dates):
    for cell in notebook_data["cells"]:
        if cell["cell_type"] == "markdown":
            cell_content = ''.join(cell["source"])
            for date in dates:
                anchor_tag = f'<a id="date-{date.replace("-", "-")}"></a>{date}'
                cell_content = cell_content.replace(date, anchor_tag)
            cell["source"] = cell_content.splitlines(True)
    return notebook_data



In [101]:
def generate_date_links(dates):
    sorted_dates = sorted(dates, key=lambda x: datetime.strptime(x, "%d-%m-%Y"))
    date_links = []
    for date in sorted_dates:
        date_links.append(f'<li><a href="#date-{date}" style="color:#333; font-style: italic;">{date}</a></li>\n')
    return date_links


In [102]:
def insert_date_links_in_template(notebook_data, date_links):
    date_links_str = ''.join(date_links)  # Join the date_links list into a single string
    for cell in notebook_data["cells"]:
        if cell["cell_type"] == "markdown":
            cell_content = ''.join(cell["source"])
            if "<li><a href=\"\" style=\"color:#333; font-style: italic;\"></a></li>" in cell_content:
                cell_content = cell_content.replace("<li><a href=\"\" style=\"color:#333; font-style: italic;\"></a></li>", date_links_str)
                cell["source"] = cell_content.splitlines(True)
                break
    return notebook_data


In [103]:
def insert_date_links_in_template0(notebook_data, date_links):
    for cell in notebook_data["cells"]:
        if cell["cell_type"] == "markdown":
            cell_content = ''.join(cell["source"])
            if "<li><a href=\"\" style=\"color:#333; font-style: italic;\"></a></li>" in cell_content:
                cell_content = cell_content.replace("<li><a href=\"\" style=\"color:#333; font-style: italic;\"></a></li>", date_links)
                cell["source"] = cell_content.splitlines(True)
                break
    return notebook_data


In [104]:
dates = extract_dates_from_cells(notebook_data)  # Extract dates
notebook_data_with_anchor_tags = generate_anchor_tags(notebook_data, dates)  # Generate anchor tags
date_links = generate_date_links(dates)  # Generate date links
notebook_data_with_date_links = insert_date_links_in_template(notebook_data_with_anchor_tags, date_links)  # Insert date links into the template

# Save the updated notebook data to a new Jupyter Notebook file
with open("Ali-diary-notebook.ipynb", "w") as f:
    json.dump(notebook_data_with_date_links, f)
